In [1]:
from random import *
from pulp import *

In [2]:
n, low_val, high_val = 10, 50, 100
days, band = 200, 30

In [3]:
values = [0] + [randint(low_val,high_val) for i in xrange(n)]
prices = [[v + randint(-band*10000/2,band*10000/2)/10000.0 for v in values] for day in xrange(days)]
profits = [[values[i] - p_t[i] for i in xrange(n)] for p_t in prices]
max_profit_items = [profit.index(max(profit)) for profit in profits]
T_i = [[d for d in xrange(days) if max_profit_items[d] == i] for i in xrange(n)]
w_ij = [[min([prices[d][j] - prices[d][i] for d in T_i[i]]) for j in xrange(n)] for i in xrange(n)]

In [4]:
prob = LpProblem("primal", LpMaximize)
Primal_Values = LpVariable.dicts("PrimalValue",[pv for pv in xrange(n)],0)
prob += lpSum([Primal_Values[x] for x in xrange(1,n)]), "Total Value"
for i in xrange(n):
    for j in xrange(n):
        if i != j:
            prob += lpSum([Primal_Values[j], -Primal_Values[i]]) <= w_ij[i][j], "w_{0}{1}".format(i,j)
prob += Primal_Values[0] == 0, "v0 = 0"
prob.solve()
print values
print LpStatus[prob.status]
print value(prob.objective)
for v in prob.variables():
    print v.name, "=", v.varValue

[0, 93, 61, 52, 62, 68, 99, 70, 63, 69, 63]
Optimal
640.1458
PrimalValue_0 = 0.0
PrimalValue_1 = 93.4263
PrimalValue_2 = 61.4048
PrimalValue_3 = 52.1084
PrimalValue_4 = 62.2467
PrimalValue_5 = 68.5243
PrimalValue_6 = 99.2412
PrimalValue_7 = 70.4058
PrimalValue_8 = 63.2766
PrimalValue_9 = 69.5117


In [5]:
prob_dual = LpProblem("dual", LpMinimize)
ijs = [(i,j) for i in xrange(n) for j in xrange(n) if j != i]
Dual_Values = LpVariable.dicts("DualValue",["x_{0}{1}".format(i,j) for (i,j) in ijs],0)
prob_dual += lpSum([Dual_Values["x_{0}{1}".format(i,j)] * w_ij[i][j] for (i,j) in ijs])
free_y = LpVariable("y")
prob_dual += lpSum([free_y,-1] + \
                   [Dual_Values["x_{0}0".format(j)] for j in xrange(1,n)] + \
                   [-Dual_Values["x_0{0}".format(j)] for j in xrange(1,n)]) >= 0
for i in xrange(1,n):
    prob_dual += lpSum([-1] + \
                   [Dual_Values["x_{0}{1}".format(j,i)] for j in xrange(0,n) if j != i] + \
                   [-Dual_Values["x_{1}{0}".format(j,i)] for j in xrange(0,n) if j != i]) >= 0
prob_dual.solve()
print LpStatus[prob_dual.status]
for v in prob_dual.variables():
    if v.varValue != 0.0:
        print v.name, "=", v.varValue
print value(prob_dual.objective)

Optimal
DualValue_x_02 = 3.0
DualValue_x_03 = 2.0
DualValue_x_06 = 3.0
DualValue_x_09 = 1.0
DualValue_x_15 = 1.0
DualValue_x_21 = 2.0
DualValue_x_38 = 1.0
DualValue_x_47 = 1.0
DualValue_x_64 = 2.0
y = 10.0
640.1458
